In [17]:
import os
import json
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [18]:
root_path = Path('./data/')
n_optim = 50
results = {}

for subdir, dirs, files in os.walk(root_path):
    for file in files:
        if file == 'results.json':
            print(os.path.join(subdir, file))
            _, query, params, seed, budget = subdir.split('/')
            if query not in results:
                results[query] = {}

            if params not in results[query]:
                results[query][params] = {}

            if seed not in results[query][params]:
                results[query][params][seed] = {}

            with open(subdir + '/' + 'results.json', 'r') as f:
                res = json.load(f)
            results[query][params][seed][budget] = res

data/badge/lr0.001_wd0.05/seed1/budget2000/results.json
data/badge/lr0.001_wd0.05/seed1/budget4000/results.json
data/badge/lr0.001_wd0.05/seed3/budget2000/results.json
data/badge/lr0.001_wd0.05/seed2/budget2000/results.json
data/badge/lr0.001_wd0.05/seed2/budget4000/results.json
data/badge/lr0.01_wd0.0005/seed1/budget2000/results.json
data/badge/lr0.01_wd0.0005/seed1/budget4000/results.json
data/badge/lr0.01_wd0.0005/seed3/budget2000/results.json
data/badge/lr0.01_wd0.0005/seed2/budget2000/results.json
data/badge/lr0.01_wd0.0005/seed2/budget4000/results.json
data/badge/lr0.01_wd0.05/seed1/budget2000/results.json
data/badge/lr0.01_wd0.05/seed1/budget4000/results.json
data/badge/lr0.01_wd0.05/seed3/budget2000/results.json
data/badge/lr0.01_wd0.05/seed2/budget2000/results.json
data/badge/lr0.1_wd0.005/seed1/budget2000/results.json
data/badge/lr0.1_wd0.005/seed3/budget2000/results.json
data/badge/lr0.1_wd0.005/seed2/budget2000/results.json
data/random/lr0.001_wd0.05/seed1/budget2000/result

In [19]:
# Analyse Results

for query in results:
    for params in results[query]:
        for seed in results[query][params]:
            for budget in results[query][params][seed]:
                print(' \t Query:',query, '  \t Params:' , params, ' \t Seed:' , seed[-1] , ' \t Budget:' , budget[-4:] , ' \t Number of trials: ' , len(results[query][params][seed][budget]['ray-results']) , ' \t Avg. Runtime per Trial:' , np.mean([t['res']['time_total_s'] for t in results[query][params][seed][budget]['ray-results']]).round(2) , ' \t Max. Runtime per Trial:' , np.max([t['res']['time_total_s'] for t in results[query][params][seed][budget]['ray-results']]).round(2) , ' \t Avg. Final Train Acc:' , str(np.mean([t['res']['train_acc']*100 for t in results[query][params][seed][budget]['ray-results']]).round(2))+" %" ,  '  \t Avg. Final Val Acc:' , str(np.mean([t['res']['val_acc']*100 for t in results[query][params][seed][budget]['ray-results']]).round(2))+" %")

 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 1  	 Budget: 2000  	 Number of trials:  100  	 Avg. Runtime per Trial: 585.42  	 Max. Runtime per Trial: 607.88  	 Avg. Final Train Acc: 57.84 %   	 Avg. Final Val Acc: 39.17 %
 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 1  	 Budget: 4000  	 Number of trials:  100  	 Avg. Runtime per Trial: 1157.6  	 Max. Runtime per Trial: 1206.86  	 Avg. Final Train Acc: 30.25 %   	 Avg. Final Val Acc: 23.93 %
 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 3  	 Budget: 2000  	 Number of trials:  100  	 Avg. Runtime per Trial: 303.89  	 Max. Runtime per Trial: 314.05  	 Avg. Final Train Acc: 47.41 %   	 Avg. Final Val Acc: 33.57 %
 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 2  	 Budget: 2000  	 Number of trials:  100  	 Avg. Runtime per Trial: 305.49  	 Max. Runtime per Trial: 316.25  	 Avg. Final Train Acc: 57.28 %   	 Avg. Final Val Acc: 39.09 %
 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 2  	 Budget: 4000  	 Number of 

In [20]:
test_accuracys = {}

for query in results:
    test_accuracys[query] = {}
    for params in results[query]:
        test_accuracys[query][params] = {}
        for budget in ['budget2000', 'budget4000']:
            test_accuracys[query][params][budget] = []
            for seed in results[query][params]:
                if budget in results[query][params][seed]:
                    # Find optimal run based on validation accuracy
                    best_val_acc = -1
                    according_test_acc = 0
                    for trial in results[query][params][seed][budget]['ray-results'][:n_optim]:
                        if trial['res']['val_acc'] > best_val_acc:
                            best_val_acc = trial['res']['val_acc']
                            according_test_acc = trial['res']['test_acc']
                    test_accuracys[query][params][budget].append(according_test_acc*100)

In [21]:
# Validate that each param group has 3 finished runs
unfinished_runs = []
for query in test_accuracys:
    for params in test_accuracys[query]:
        for budget in test_accuracys[query][params]:
            if len(test_accuracys[query][params][budget]) != 3:
                print("Assertion: The run with query strategy "+str(query)+ " and parameters "+params+" and budget "+budget+" has not had 3 finished runs yet!")
                unfinished_runs.append([query, params, budget])
                
# Remove unfinished runs
for query, params, budget in unfinished_runs:
    test_accuracys[query][params].pop(budget)


Assertion: The run with query strategy badge and parameters lr0.001_wd0.05 and budget budget4000 has not had 3 finished runs yet!
Assertion: The run with query strategy badge and parameters lr0.01_wd0.0005 and budget budget4000 has not had 3 finished runs yet!
Assertion: The run with query strategy badge and parameters lr0.01_wd0.05 and budget budget4000 has not had 3 finished runs yet!
Assertion: The run with query strategy badge and parameters lr0.1_wd0.005 and budget budget4000 has not had 3 finished runs yet!


In [22]:
df = {}
for query in test_accuracys:
    for budget in test_accuracys[query][params]:
        key = query+budget[-4:]
        df[key] = {}
        for params in test_accuracys[query]:
            df[key][params] = str(np.mean(test_accuracys[query][params][budget]).round(2)) + ' +/- ' + str(np.std(test_accuracys[query][params][budget]).round(2))
df = pd.DataFrame(df)
df.to_csv('./result_tables/results.csv')
df

,badge2000,random2000,random4000
lr0.001_wd0.05,77.42 +/- 0.76,74.23 +/- 1.65,81.96 +/- 1.33
lr0.01_wd0.0005,75.25 +/- 1.05,76.2 +/- 0.7,82.86 +/- 0.36
lr0.01_wd0.05,76.25 +/- 1.17,75.97 +/- 0.82,82.54 +/- 0.05
lr0.1_wd0.005,78.38 +/- 0.57,75.13 +/- 0.49,82.59 +/- 0.54


In [23]:
# Lets compare runtimes and number of trials

for query in results:
    for params in results[query]:
        for seed in results[query][params]:
            for budget in results[query][params][seed]:
                print(' \t Query:',query, '  \t Params:' , params, ' \t Seed:' , seed[-1] , ' \t Budget:' , budget[-4:] , ' \t Number of trials: ' , len(results[query][params][seed][budget]['ray-results']) , ' \t Avg. Runtime per Trial:' , np.mean([t['res']['time_total_s'] for t in results[query][params][seed][budget]['ray-results']]).round(2) , ' \t Avg. Final Train Acc:' , str(np.mean([t['res']['train_acc']*100 for t in results[query][params][seed][budget]['ray-results']]).round(2))+" %" ,  '  \t Avg. Final Val Acc:' , str(np.mean([t['res']['val_acc']*100 for t in results[query][params][seed][budget]['ray-results']]).round(2))+" %")


 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 1  	 Budget: 2000  	 Number of trials:  100  	 Avg. Runtime per Trial: 585.42  	 Avg. Final Train Acc: 57.84 %   	 Avg. Final Val Acc: 39.17 %
 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 1  	 Budget: 4000  	 Number of trials:  100  	 Avg. Runtime per Trial: 1157.6  	 Avg. Final Train Acc: 30.25 %   	 Avg. Final Val Acc: 23.93 %
 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 3  	 Budget: 2000  	 Number of trials:  100  	 Avg. Runtime per Trial: 303.89  	 Avg. Final Train Acc: 47.41 %   	 Avg. Final Val Acc: 33.57 %
 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 2  	 Budget: 2000  	 Number of trials:  100  	 Avg. Runtime per Trial: 305.49  	 Avg. Final Train Acc: 57.28 %   	 Avg. Final Val Acc: 39.09 %
 	 Query: badge   	 Params: lr0.001_wd0.05  	 Seed: 2  	 Budget: 4000  	 Number of trials:  100  	 Avg. Runtime per Trial: 594.86  	 Avg. Final Train Acc: 41.44 %   	 Avg. Final Val Acc: 30.55 %
 	 Query: badge   	 Param